In [ ]:
#!/usr/bin/env julia
using ArgParse

using LilGuys
using CairoMakie

using StatsBase
using PythonCall
agama = pyimport("agama")
using Printf
using LinearAlgebra: cross


In [ ]:
color_0 = Makie.RGBA{Float32}(1.0f0, 1.0f0, 1.0f0, 1.0f0)
color_1 = Makie.RGBA{Float32}(0.9411765f0,0.89411765f0,0.25882354f0,1.0f0)
color_0, color_1

In [ ]:
cd(ENV["DWARFS_ROOT"] * "/analysis/sculptor/1e6_V31_r4.2/vasiliev+21_smallperi")

In [ ]:
out = Output(".")

In [ ]:
args = Dict()
args["limits"] = 400 .* [-1, 1, -1, 1]
args["n_bins"] = 501
args["skip"] = 20

In [ ]:
potential = agama.Potential(ENV["DWARFS_ROOT"] * "/simulations/sculptor/1e6_V31_r4.2/vasiliev+21_smallperi/agama_potential.ini")


In [ ]:
xbins = LinRange(args["limits"][1], args["limits"][2], args["n_bins"])
ybins = LinRange(args["limits"][3], args["limits"][4], args["n_bins"])
bins = (xbins, ybins)
@assert issorted(xbins) && issorted(ybins)


In [ ]:
function project_agama_potential(potential, bins; x_vec=[0,1,0], y_vec=[0,0,1], time=0)
    xbins, ybins = bins
    xm, ym = midpoints(xbins), midpoints(ybins)
    Nx, Ny = length(xm), length(ym)
    
    x = repeat(xm, outer=Ny)
    y = repeat(ym, inner=Nx)

    println(size(x))
    println(size(y))
    alpha, beta, gamma = vectors_to_euler_angles(x_vec, y_vec)
    pos = [x y]
    
    println(size(pos))
    density = potential.projectedDensity(pos, alpha=alpha, beta=beta, gamma=gamma)

    Σ_disk = pyconvert(Vector{Float64}, density)
    
    return reshape(Σ_disk, (Nx, Ny))
end


function vectors_to_euler_angles(xhat::Vector{<:Real}, yhat::Vector{<:Real})
    # Compute the zhat vector
    zhat = cross(xhat, yhat)

    # Construct the rotation matrix
    R = hcat(xhat, yhat, zhat)

    # Extract Euler angles for ZYX convention
    alpha = atan(-R[3,2], R[3,1])
    beta = acos(R[3,3])
    gamma = atan(R[2,3], R[1,3])

    return alpha, beta, gamma
end

In [ ]:
function project_agama_potential(potential, bins; x_vec=[0,1,0], y_vec=[0,0,1], 
        time=0, cutoff=200, cutoffstrength=3)
    # takes a matrix of positions, returns array of densities for each position
    calc_ρ(x) = pyconvert(Vector{Float64}, potential.density(x, t=time))

    xbins, ybins = bins
    xm, ym = midpoints(xbins), midpoints(ybins)
    Nx, Ny = length(xm), length(ym)

    # TODO: cludge...
    zbins = xbins
    zm = midpoints(zbins)
    Nz = length(zm)

    z_vec = cross(x_vec, y_vec)

    Σ_disk = Matrix{Float64}(undef, Nx, Ny)

    for i in 1:Nx # iterate over each x coordinate
        x = fill(xm[i], Ny*Nz)
        # y and z are vectorized so less python calls
        y = repeat(ym, outer=Nz)
        z = repeat(zm, inner=Ny)
        r = @. sqrt(x^2 + y^2 + z^2)
        pos = @. x' * x_vec + y' * y_vec + z' * z_vec
        ρ = calc_ρ(pos')

        if cutoff !== nothing
            ρ = @. ρ * exp(-(r/cutoff)^cutoffstrength)
        end
        ρ = reshape(ρ, Ny, Nz)
        
        Σ = sum(ρ, dims=2) # sum over Z dimension
        Σ_disk[i, :] = Σ
    end

    return Σ_disk
end


In [ ]:
function project_points(x, y, xybins)
    h1 = fit(Histogram, (x, y), xybins )
end

function I_to_SI(I)
    if isnan(I)
        return 0.
    end
    if I === Inf
        return 1.
    end
    
    @assert 0 <= I <= 1 "$I is out of range"

    if I < 0.03928 / 12.92
        return I * 12.92
    else
        return I^(1/2.4) * 1.055 - 0.055
    end
end

function normalize_colors(colors)
    if isnan(colors)
        return 0.0
    elseif colors === Inf
        return 1.0
    end
    colors = min(colors, 1.0)
    colors = max(colors, 0.0)
    
    return colors
end

function map_intensities(A, B, color_A, color_B)
    R = @. A * Makie.red(color_A) + B * Makie.red(color_B)
    G = @. A * Makie.green(color_A) + B * Makie.green(color_B)
    B = @. A * Makie.blue(color_A) + B * Makie.blue(color_B)
    R = normalize_colors.(R) 
    G = normalize_colors.(G) 
    B = normalize_colors.(B)
    color_R = @. I_to_SI.(R)
    color_G = @. I_to_SI.(G)
    color_B = @. I_to_SI.(B)

    
    return RGBf.(color_R, color_G, color_B)
end

function map_intensities(A, color_A;)
    color_R = I_to_SI.(normalize_colors.(A .* Makie.red.(color_A)))
    color_G = I_to_SI.(normalize_colors.(A .* Makie.green.(color_A)))
    color_B = I_to_SI.(normalize_colors.(A .* Makie.blue.(color_A)))

    
    return RGBf.(color_R, color_G, color_B)
end

In [ ]:
Sigma_mw = project_agama_potential(potential, bins)

In [ ]:
map_intensities((max.(Sigma_mw ./ maximum(Sigma_mw[isfinite.(Sigma_mw)]), 0)) .^ (1),color_0)

In [ ]:
Sigma_mw = project_agama_potential(potential, bins, time=-5)

In [ ]:
map_intensities((max.(Sigma_mw ./ maximum(Sigma_mw[isfinite.(Sigma_mw)]), 0)) .^ 0.3,color_1, )

In [ ]:

function get_xy(out, idx; x_vec=[0, 1, 0], y_vec = [0, 0, 1])
    # shortcut for hdf5
    # -1 since HDF5 is zero indexed
    idx -= 1
    pos = out.h5file["snap$idx/PartType1/Coordinates"][:, :]
    A = [x_vec y_vec]'
    xy = A * pos
    return xy[1, :], xy[2, :]
end

In [ ]:

function animate(out, bins, animation_dir; 
        idx=eachindex(out), 
        x_vec=[0, 1, 0], y_vec = [0, 0, 1], 
        colorrange=nothing,
        potential=nothing, evolving_potential=false, potential_scale=1.0,
        scalebar=50, font="Arial"
    )

    if colorrange == nothing
        x, y = get_xy(out, 1; x_vec=x_vec, y_vec=y_vec)
        h = project_points(x, y, bins)
        colorrange = maximum(h.weights)
    end

    @info "using colorrange: $colorrange"

    if potential !== nothing
        @info "projecting potential"
        Sigma_mw = project_agama_potential(potential, bins; x_vec=x_vec, y_vec=y_vec)
        potential_scale /= maximum(Sigma_mw[isfinite.(Sigma_mw)])
        Sigma_mw .*= potential_scale
    end

    for (n, i) in enumerate(idx)
        print("creating frame $n / $(length(idx))\r")
        if evolving_potential && potential !== nothing
            Sigma_mw = project_agama_potential(potential, bins; x_vec=x_vec, y_vec=y_vec, time=i)
            Sigma_mw .*= potential_scale
        end

        x, y = get_xy(out, i; x_vec=x_vec, y_vec=y_vec)
        h = project_points(x, y, bins)
        fig, ax, p = make_frame(h, colorrange=colorrange, scalebar=scalebar,
            font=font, Sigma_mw=Sigma_mw
        )
        Makie.save(joinpath(animation_dir, "frame_$i.png"), fig)
    end
end


function make_frame(h; colorrange, font, scalebar=50, Sigma_mw=nothing,
        color_A=color_0, color_B=color_1,
        dm_power = 1/2
    )
    fig = Figure(figure_padding=(0,0,0,0), size=(200,200), backgroundcolor=:black) # px scale is x5 so this works for 1000 bins
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
    )
    
    xrange, yrange = extrema.(h.edges)

    Sigma_dm = (h.weights ./ colorrange) .^ dm_power

    if Sigma_mw !== nothing
        Sigma_mw = Sigma_mw 
        colors = map_intensities(Sigma_dm, Sigma_mw, color_A, color_B)
    else
        colors = map_intensities(Sigma_dm, color_A)
    end

    p = image!(xrange, yrange, colors)

    hidespines!(ax)
    resize_to_layout!(fig)
    hidedecorations!(ax)

    # create scalebar

    x1 = xrange[1] + scalebar / 2
    y1 = yrange[1] + scalebar / 2
    x2 = x1 + scalebar
    y2 = y1

    lines!([x1, x2], [y1, y2], color=:grey)
    text!(x1, y1, text="$scalebar kpc", color=:grey, font=font, fontsize=7.5)

    return Makie.FigureAxisPlot(fig, ax, p)
end









In [ ]:
x, y = get_xy(out, 200)

In [ ]:
h = project_points(x, y, bins)

In [ ]:
maximum(Sigma_mw[isfinite.(Sigma_mw)])

In [ ]:
make_frame(h, colorrange=maximum(h.weights), font="areal", Sigma_mw= (Sigma_mw) ./ maximum(Sigma_mw[isfinite.(Sigma_mw)]))



In [ ]:
animate(out, bins, "dm_animation"; 
        idx=eachindex(out), 
        potential=potential,
       )